# Evolver Loop 14 - LB Feedback Analysis

## Current Status
- **Best CV**: 70.3295 (exp_017)
- **Best LB**: 70.3295 (confirmed!)
- **Target**: 68.876781
- **Gap**: 1.45 points (2.1%)

## Key Observations
1. CV-LB alignment is PERFECT (gap = 0.0000)
2. The rebuild from corners technique found 0 improvements
3. Evaluator identified a CRITICAL BUG in exp_017

In [1]:
# Analyze the bug in exp_017
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# Tree shape vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def create_tree_polygon(x, y, angle):
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

# Demonstrate the bug
print("BUG DEMONSTRATION:")
print("="*60)

# Tree at center (0,0) with 45 degree rotation
x, y, angle = 0, 0, 45
poly = create_tree_polygon(x, y, angle)
bounds = poly.bounds  # (minx, miny, maxx, maxy)

print(f"Tree at center ({x}, {y}) with angle {angle}°")
print(f"Tree center: ({x}, {y})")
print(f"Polygon bounds: minx={bounds[0]:.4f}, miny={bounds[1]:.4f}, maxx={bounds[2]:.4f}, maxy={bounds[3]:.4f}")
print(f"\nBUG: exp_017 used tree center ({x}, {y}) for distance")
print(f"CORRECT: Should use polygon bounds which extend to ±{max(abs(bounds[0]), abs(bounds[2])):.4f}")
print(f"\nDifference: {max(abs(bounds[0]), abs(bounds[2])) - abs(x):.4f}")

BUG DEMONSTRATION:
Tree at center (0, 0) with angle 45°
Tree center: (0, 0)
Polygon bounds: minx=-0.5657, miny=-0.2475, maxx=0.2475, maxy=0.5657

BUG: exp_017 used tree center (0, 0) for distance
CORRECT: Should use polygon bounds which extend to ±0.5657

Difference: 0.5657


In [2]:
# Load current best and analyze score breakdown
df = pd.read_csv('/home/code/experiments/016_jazivxt_ensemble/submission.csv')
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

def get_bbox_side(trees):
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    side = get_bbox_side(trees)
    return (side ** 2) / n

# Calculate per-N scores
scores_by_n = {}
for n, g in df.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    scores_by_n[n] = get_score(trees, n)

# Score breakdown by range
ranges = [(1, 1), (2, 5), (6, 10), (11, 50), (51, 100), (101, 200)]
print("Score breakdown by N range:")
print("="*60)
for start, end in ranges:
    range_score = sum(scores_by_n[n] for n in range(start, end+1))
    print(f"N={start}-{end}: {range_score:.4f}")

total = sum(scores_by_n.values())
print(f"\nTotal: {total:.6f}")
print(f"Target: 68.876781")
print(f"Gap: {total - 68.876781:.6f}")

Score breakdown by N range:
N=1-1: 0.6612
N=2-5: 1.7189
N=6-10: 1.9446
N=11-50: 14.6300
N=51-100: 17.4810
N=101-200: 33.8938

Total: 70.329514
Target: 68.876781
Gap: 1.452733


In [3]:
# Identify which N values have the most room for improvement
# Compare to theoretical minimum (area-based lower bound)

print("\nPer-N analysis - largest contributors:")
print("="*60)

# Sort by score contribution
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)

print("Top 20 N values by score contribution:")
for n, score in sorted_scores[:20]:
    print(f"  N={n}: {score:.6f}")

print("\nBottom 20 N values by score contribution:")
for n, score in sorted_scores[-20:]:
    print(f"  N={n}: {score:.6f}")


Per-N analysis - largest contributors:
Top 20 N values by score contribution:
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399842
  N=6: 0.399610
  N=8: 0.385407
  N=9: 0.383125
  N=10: 0.376630
  N=11: 0.374921
  N=15: 0.374381
  N=12: 0.372724
  N=13: 0.372267
  N=20: 0.371795
  N=16: 0.370191
  N=21: 0.370154
  N=17: 0.370040
  N=22: 0.369821
  N=14: 0.369543

Bottom 20 N values by score contribution:
  N=163: 0.335975
  N=197: 0.335621
  N=165: 0.335528
  N=192: 0.335074
  N=166: 0.334819
  N=178: 0.334282
  N=154: 0.334144
  N=132: 0.333775
  N=193: 0.333339
  N=196: 0.333261
  N=194: 0.332999
  N=167: 0.332835
  N=195: 0.332576
  N=179: 0.332496
  N=168: 0.332475
  N=155: 0.332069
  N=180: 0.330796
  N=182: 0.329988
  N=156: 0.329984
  N=181: 0.329259


## Key Insight from Evaluator

The rebuild from corners technique was NOT properly tested due to a critical bug:

**BUG**: Used tree CENTER for distance calculation
```python
dist = max(abs(x - corner_x), abs(y - corner_y))  # WRONG!
```

**CORRECT** (from chistyakov kernel):
```python
dist = max(
    abs(tree.polygon.bounds[0] - corner_x),  # polygon minx
    abs(tree.polygon.bounds[2] - corner_x),  # polygon maxx  
    abs(tree.polygon.bounds[1] - corner_y),  # polygon miny
    abs(tree.polygon.bounds[3] - corner_y),  # polygon maxy
)
```

A tree at center (0,0) with 45° rotation has polygon bounds extending to ±0.7!
Using tree center ignores rotation and shape, causing wrong tree selection.

In [4]:
# Check what other approaches haven't been tried
print("\nAPPROACHES TRIED:")
print("="*60)
approaches = [
    ("Ensemble from snapshots", "EXHAUSTED - 3781 snapshots mined"),
    ("Ensemble from Kaggle datasets", "WORKING - main source of improvement"),
    ("Simulated annealing", "FAILED - 0 improvements"),
    ("Rotation optimization", "FAILED - 0 improvements"),
    ("Fractional translation", "MARGINAL - 0.000033 improvement"),
    ("NFP local search", "FAILED - 0 improvements"),
    ("Rebuild from corners", "BUGGY - needs re-test with fix"),
]

for approach, status in approaches:
    print(f"  {approach}: {status}")

print("\nAPPROACHES NOT YET TRIED:")
print("="*60)
not_tried = [
    "Fix rebuild from corners bug and re-run",
    "Download more Kaggle datasets (jonathanchan uses 19 sources)",
    "Genetic algorithm with custom crossover",
    "Branch-and-bound for small N (N=1-20)",
    "Tessellation patterns for large N",
    "Asymmetric solutions (per discussion)",
]
for approach in not_tried:
    print(f"  - {approach}")


APPROACHES TRIED:
  Ensemble from snapshots: EXHAUSTED - 3781 snapshots mined
  Ensemble from Kaggle datasets: WORKING - main source of improvement
  Simulated annealing: FAILED - 0 improvements
  Rotation optimization: FAILED - 0 improvements
  Fractional translation: MARGINAL - 0.000033 improvement
  NFP local search: FAILED - 0 improvements
  Rebuild from corners: BUGGY - needs re-test with fix

APPROACHES NOT YET TRIED:
  - Fix rebuild from corners bug and re-run
  - Download more Kaggle datasets (jonathanchan uses 19 sources)
  - Genetic algorithm with custom crossover
  - Branch-and-bound for small N (N=1-20)
  - Tessellation patterns for large N
  - Asymmetric solutions (per discussion)
